In [13]:
import pandas as pd
from thefuzz import fuzz, process

In [14]:
# Đọc danh sách các country theo tên chuẩn lấy từ GeoNames
countries = pd.read_csv('../data/country.csv')["Country"]

In [15]:
# Đọc các tập dữ liệu
co2_df = pd.read_csv('../data/co2_cleaned.csv').rename(columns={"value": "co2"})
climate_change_df = pd.read_csv('../data/climate_change_cleaned.csv').rename(columns={"value": "climate_change"})
energy_df = pd.read_csv('../data/energy_cleaned.csv').rename(columns={"value": "energy"})

In [16]:
def find_best_match(name):
  best_match = process.extractOne(name, countries, scorer=fuzz.WRatio)
  return best_match[0], best_match[1]

In [17]:
co2_df_country = co2_df.loc[:, ["country"]].drop_duplicates(subset="country")

In [18]:
energy_df_country = energy_df.loc[:, ["country"]].drop_duplicates(subset="country")

In [19]:
climate_change_df_country =climate_change_df.loc[:, ["country"]].drop_duplicates(subset="country")

In [20]:
matches = co2_df_country["country"].apply(find_best_match)
co2_df_country[["normalized_country", 'match_score']] = pd.DataFrame(matches.to_list(), index=co2_df_country.index)

In [21]:
matches = energy_df_country["country"].apply(find_best_match)
energy_df_country[["normalized_country", 'match_score']] = pd.DataFrame(matches.to_list(), index=energy_df_country.index)

In [22]:
matches = climate_change_df_country["country"].apply(find_best_match)
climate_change_df_country[["normalized_country", 'match_score']] = pd.DataFrame(matches.to_list(), index=climate_change_df_country.index)

In [23]:
co2_df_country[co2_df_country["match_score"] < 100].sort_values(by="match_score")
# kiểm tra match_score truớc
# bỏ global, international
# với mấy cái điểm thấp thì mapping bằng tay, ví dụ USA -> United State

,country,normalized_country,match_score
54800,Türkiye,Turkey,67
9590,Cape Verde,Cabo Verde,80
7946,Brunei Darussalam,Brunei,90
11782,Congo,Democratic Republic of the Congo,90
18084,Micronesia (Federated States of),Micronesia,90
28222,Kuwaiti Oil Fires,Kuwait,90
39730,Pacific Islands (Palau),Palau,90
46032,Sint Maarten (Dutch part),Sint Maarten,90
13426,Curaçao,Curacao,92
58636,Viet Nam,Vietnam,93


In [24]:
energy_df_country[energy_df_country["match_score"] < 100].sort_values(by="match_score")

,country,normalized_country,match_score
9932,World,Jordan,55
6192,Non-OPEC,British Indian Ocean Territory,56
6412,OECD - Europe,Peru,60
8172,Middle East,Mali,60
8436,Non-OECD,Chad,60
2068,OECD,Ecuador,60
6016,Europe,Guadeloupe,60
2244,IEO - Middle East,Niue,60
10592,Other Asia-Pacific,Austria,62
5312,Western Europe,Western Sahara,64


In [25]:
climate_change_df_country[climate_change_df_country["match_score"] < 100].sort_values(by="match_score")

,country,normalized_country,match_score
221,World,Jordan,55
86,Holy See,French Polynesia,56
107,Kyrgyz Rep.,Kyrgyzstan,60
52,Czech Rep.,Czechia,62
178,Slovak Rep.,Slovakia,67
...,...,...,...
64,"Ethiopia, The Federal Dem. Rep. of",Ethiopia,90
222,"Yemen, Rep. of",Yemen,90
188,St. Vincent and the Grenadines,Saint Vincent and the Grenadines,94
141,"Netherlands, The",The Netherlands,95


In [26]:
# Chuẩn hóa lại các quốc gia hàm chuẩn hóa chưa thực hiện được
def normalize_country_name(df, column_name):

    country_mapping = {
        "USA": "United States",
        "Côte d'Ivoire": "Ivory Coast",
        "Ryukyu Islands": "Japan",
        "State of Palestine": "Palestinian Territory",
        "Holy See": "Italy"
    }
    # Thay thế tên quốc gia dựa trên từ điển
    df[column_name] = df[column_name].replace(country_mapping)
    return df

# Áp dụng hàm 
co2_df = normalize_country_name(co2_df, "country")
energy_df = normalize_country_name(energy_df, "country")
climate_change_df = normalize_country_name(climate_change_df, "country")

# Ghi lại các DataFrame đã xử lý vào các file CSV mới
co2_df.to_csv('../data/co2_cleaned.csv', index=False)
energy_df.to_csv('../data/energy_cleaned.csv', index=False)
climate_change_df.to_csv('../data/climate_change_cleaned.csv', index=False)

In [28]:
# Loại bỏ Global/International
def remove_global_entries(df, column_name):
    # Lọc các hàng không có giá trị 'Global' và không chứa 'International'
    return df[~df[column_name].str.contains("International|World", na=False) & (df[column_name] != "Global")]

# Áp dụng hàm
co2_df = remove_global_entries(co2_df, "country")
energy_df = remove_global_entries(energy_df, "country")
climate_change_df = remove_global_entries(climate_change_df, "country")

# Ghi lại các DataFrame đã xử lý vào các file CSV mới
co2_df.to_csv('../data/co2_cleaned.csv', index=False)
energy_df.to_csv('../data/energy_cleaned.csv', index=False)
climate_change_df.to_csv('../data/climate_change_cleaned.csv', index=False)
